In [11]:
from __future__ import division

import contextlib
import os, platform, sys, traceback
from pprint import pformat, pprint
import numpy as np
from climin.util import optimizer
from itertools import repeat, cycle, islice, izip
import random

from schlichtanders.mycontextmanagers import ignored

inf = float("inf")

from schlichtanders.myfunctools import compose, meanmap, summap, compose_fmap, Average
from schlichtanders.mygenerators import eatN, chunk, chunk_list, every, takeN
from schlichtanders.myobjects import NestedNamespace, Namespace

import theano_models as tm
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
from theano_models import data

from sklearn import cross_validation
from theano.tensor.shared_randomstreams import RandomStreams
import theano

from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

tm.inputting_references.update(['to_be_randomized'])
tm.inputting_references, tm.outputting_references

EPS = 1e-8

pm.RNG = NestedNamespace(tm.PooledRandomStreams(pool_size=int(5e8)), RandomStreams())

__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

suffix = ""
datasetname = "boston"

class Track(object):
    def __getattr__(self, item):
        return tm.track_model(getattr(tm, item))
track = Track()

# # Data
#     # datasetnames = ["boston", "concrete", "energy", "kin8nm", "naval", "powerplant", "protein", "winered", "yacht", "year"]
#     datasetnames = ["boston", "concrete", "energy", "kin8nm", "naval", "powerplant", "winered", "yacht"]
# datasetname = "concrete"

# TODO check planar flows, they don't work as expected... however radial flows work.. it is weird

In [12]:
Z, X = getattr(data, "_" + datasetname)()
# normalization is standard in Probabilistic Backpropagation Paper
X_mean = X.mean(0)
X_std = X.std(0)
X = (X - X_mean) / X_std
Z_mean = Z.mean(0)
Z_std = Z.std(0)
Z = (Z - Z_mean) / Z_std

X, TX, Z, TZ = cross_validation.train_test_split(X, Z, test_size=0.1) # 10% test used in paper
X, VX, Z, VZ = cross_validation.train_test_split(X, Z, test_size=0.1) # 20% validation used in paper

@contextlib.contextmanager
def log_exceptions(title, *exceptions):
    if not exceptions:
        exceptions = Exception
    try:
        yield
    except exceptions:
        with open(os.path.join(__path__, 'experiment%s_errors.txt' % suffix), "a") as myfile:
            error = """
%s
------------
LAST HYPER: %s
ORIGINAL ERROR: %s""" % (title, pformat(hyper.__dict__), traceback.format_exc())
            myfile.write(error)


def RMSE(PX, Z):
    return np.sqrt(((PX - Z) ** 2).mean())

def nRMSE(PX, Z):
    return RMSE(PX*Z_std + Z_mean, Z*Z_std + Z_mean)

In [13]:
# # Hyperparameters

engine = create_engine('sqlite:///' + os.path.join(__path__, 'test%s.db' % suffix))
Base = declarative_base(bind=engine)

class RandomHyper(Base):
    __tablename__ = "hyper"
    id = Column(Integer, primary_key=True)


    # hyper parameters:
    datasetname = Column(String)
    max_epochs_without_improvement = Column(Integer)
    logP_average_n = Column(Integer)
    errorrate_average_n = Column(Integer)
    units_per_layer = Column(Integer)
    minus_log_s = Column(Integer)
    batch_size = Column(Integer)
    
    n_normflows = Column(Integer)
    
    opt_identifier = Column(String)
    opt_momentum = Column(Float)
    opt_offset = Column(Float)
    opt_decay = Column(Float)
    opt_step_rate = Column(Float)

    # baseline:
    baseline_best_val_loss = Column(Float)
    baseline_best_parameters = Column(PickleType, nullable=True)
    baseline_train_loss = Column(PickleType)
    baseline_val_loss = Column(PickleType)
    baseline_epochs = Column(Integer)
    baseline_init_params = Column(PickleType, nullable=True)
    baseline_val_error_rate = Column(Float)

    # planarflow:
    planarflow_best_val_loss = Column(Float)
    planarflow_best_parameters = Column(PickleType, nullable=True)
    planarflow_train_loss = Column(PickleType)
    planarflow_val_loss = Column(PickleType)
    planarflow_epochs = Column(Integer)
    planarflow_init_params = Column(PickleType, nullable=True)
    planarflow_val_error_rate = Column(Float)

    # planarflow deterministic:
    planarflowdet_best_val_loss = Column(Float)
    planarflowdet_best_parameters = Column(PickleType, nullable=True)
    planarflowdet_train_loss = Column(PickleType)
    planarflowdet_val_loss = Column(PickleType)
    planarflowdet_epochs = Column(Integer)
    planarflowdet_init_params = Column(PickleType, nullable=True)
    planarflowdet_val_error_rate = Column(Float)

    # planarflow maximum likelihood:
    planarflowml_best_val_loss = Column(Float)
    planarflowml_best_parameters = Column(PickleType, nullable=True)
    planarflowml_train_loss = Column(PickleType)
    planarflowml_val_loss = Column(PickleType)
    planarflowml_epochs = Column(Integer)
    planarflowml_init_params = Column(PickleType, nullable=True)
    planarflowml_val_error_rate = Column(Float)

    # radialflow:
    radialflow_best_val_loss = Column(Float)
    radialflow_best_parameters = Column(PickleType, nullable=True)
    radialflow_train_loss = Column(PickleType)
    radialflow_val_loss = Column(PickleType)
    radialflow_epochs = Column(Integer)
    radialflow_init_params = Column(PickleType, nullable=True)
    radialflow_val_error_rate = Column(Float)

    # radialflow deterministic:
    radialflowdet_best_val_loss = Column(Float)
    radialflowdet_best_parameters = Column(PickleType, nullable=True)
    radialflowdet_train_loss = Column(PickleType)
    radialflowdet_val_loss = Column(PickleType)
    radialflowdet_epochs = Column(Integer)
    radialflowdet_init_params = Column(PickleType, nullable=True)
    radialflowdet_val_error_rate = Column(Float)

    # radialflow maximum likelihood:
    radialflowml_best_val_loss = Column(Float)
    radialflowml_best_parameters = Column(PickleType, nullable=True)
    radialflowml_train_loss = Column(PickleType)
    radialflowml_val_loss = Column(PickleType)
    radialflowml_epochs = Column(Integer)
    radialflowml_init_params = Column(PickleType, nullable=True)
    radialflowml_val_error_rate = Column(Float)

    # mixture:
    mixture_best_val_loss = Column(Float)
    mixture_best_parameters = Column(PickleType, nullable=True)
    mixture_train_loss = Column(PickleType)
    mixture_val_loss = Column(PickleType)
    mixture_epochs = Column(Integer)
    mixture_init_params = Column(PickleType, nullable=True)
    mixture_val_error_rate = Column(Float)

    # mixture:
    mixtureml_best_val_loss = Column(Float)
    mixtureml_best_parameters = Column(PickleType, nullable=True)
    mixtureml_train_loss = Column(PickleType)
    mixtureml_val_loss = Column(PickleType)
    mixtureml_epochs = Column(Integer)
    mixtureml_init_params = Column(PickleType, nullable=True)
    mixtureml_val_error_rate = Column(Float)

    def __init__(self, hyper_dict=None):  # we directly refer to dict as sqlalchemy deletes the dict once committed (probably for detecting changes
        if hyper_dict is not None:
            for k, v in hyper_dict.iteritems():
                if not k.startswith("_"):
                    setattr(self, k, copy(v))
            self.init_results()
            return
        self.datasetname = datasetname
        # hyper parameters:
        self.max_epochs_without_improvement = 30
        self.batch_size = random.choice([1,10, 100])
        self.logP_average_n = 1
        self.errorrate_average_n = 10
        self.units_per_layer = 50
        self.minus_log_s = random.choice([1,2,3,4,5,6,7])
        # the prior is learned together with the other models in analogy to the paper Probabilistic Backpropagation
        
        self.n_normflows = random.choice([1,2,3,4,8,20])  #32 is to much for theano... unfortunately
        
        self.opt_identifier = random.choice(["adadelta", "adam", "rmsprop"])
        if self.opt_identifier == "adadelta":
            self.opt_momentum = random.choice([np.random.uniform(0, 0.01), np.random.uniform(0.9, 1)])
            self.opt_offset = random.choice([5e-5, 1e-8])
            self.opt_step_rate = random.choice([1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
        elif self.opt_identifier == "adam":
            self.opt_momentum = random.choice([np.random.uniform(0, 0.01), np.random.uniform(0.8, 0.93)])
            self.opt_offset = 10 ** -np.random.uniform(3, 4)
            self.opt_step_rate = random.choice([1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
        elif self.opt_identifier == "rmsprop":
            self.opt_momentum = random.choice([np.random.uniform(0.002, 0.008), np.random.uniform(0.9, 1)])
            self.opt_offset = np.random.uniform(0, 0.000045)
            self.opt_step_rate = random.choice([1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
        self.opt_decay = np.random.uniform(0.78, 1)
        
        self.init_results()
    
    def init_results(self):
        # extra for being able to reset results for loaded hyperparameters
        for prefix in ['baseline_', 'mixture_', 'mixtureml_',
                       'planarflow_', 'planarflowdet_', 'planarflowml_',
                       'radialflow_', 'radialflowdet_', 'radialflowml_']:
            setattr(self, prefix + "best_parameters", None)
            setattr(self, prefix + "best_val_loss", inf)
            setattr(self, prefix + "train_loss", [])
            setattr(self, prefix + "val_loss", [])
            setattr(self, prefix + "best_epoch", 0)
            setattr(self, prefix + "init_params ", None)
            setattr(self, prefix + "val_error_rate", inf)
            
Base.metadata.create_all()
Session = sessionmaker(bind=engine)
sql_session = Session()

In [23]:
hyper = RandomHyper()
sql_session.add(hyper)

In [24]:
hyper.opt_momentum

0.9060757143956466

In [41]:
sql_session.commit()

In [27]:
hyper.baseline_best_parameters = [1,2,3]

In [38]:
getattr(hyper, 'baseline_best_parameters')

[1, 2, 3]

In [42]:
hyper.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd88c2bb4d0>,
 'baseline_best_epoch': 0,
 'baseline_init_params ': None,
 'mixture_best_epoch': 0,
 'mixture_init_params ': None,
 'mixtureml_best_epoch': 0,
 'mixtureml_init_params ': None,
 'planarflow_best_epoch': 0,
 'planarflow_init_params ': None,
 'planarflowdet_best_epoch': 0,
 'planarflowdet_init_params ': None,
 'planarflowml_best_epoch': 0,
 'planarflowml_init_params ': None,
 'radialflow_best_epoch': 0,
 'radialflow_init_params ': None,
 'radialflowdet_best_epoch': 0,
 'radialflowdet_init_params ': None,
 'radialflowml_best_epoch': 0,
 'radialflowml_init_params ': None}

In [26]:
hyper.opt_momentum

0.9060757143956466

In [ ]:
sqlalchemy_test.py